<center><img src="ktp_logo.png" width=400 style="display: inline-block;"></center> 

# Машинное обучение
## Семинар 3. Реализация KNN и Линейной регреcсии

## 2 ноября 2023

In [137]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [138]:
data = pd.read_csv('AppleStore.csv')
data.head()

,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1


## Алгоритм действий:
1. Посмотреть на данные, разделить их на категориальные, числовые признаки, отделить колонку с ответом. Необходимо обратить внимание на выбросы и отсутствующие значения
2. Подготовка данных: чистка, масштабирование, преобразование итд. итп. 
3. Разбиение данных  на обучающую и тестовую выборки. 
4. Обучение модели
5. Оценка точности модели

## Анализ данных

In [139]:
data.head()

,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1


In [140]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7197 entries, 0 to 7196
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        7197 non-null   int64  
 1   id                7197 non-null   int64  
 2   track_name        7197 non-null   object 
 3   size_bytes        7197 non-null   int64  
 4   currency          7197 non-null   object 
 5   price             7197 non-null   float64
 6   rating_count_tot  7197 non-null   int64  
 7   rating_count_ver  7197 non-null   int64  
 8   user_rating       7197 non-null   float64
 9   user_rating_ver   7197 non-null   float64
 10  ver               7197 non-null   object 
 11  cont_rating       7197 non-null   object 
 12  prime_genre       7197 non-null   object 
 13  sup_devices.num   7197 non-null   int64  
 14  ipadSc_urls.num   7197 non-null   int64  
 15  lang.num          7197 non-null   int64  
 16  vpp_lic           7197 non-null   int64  


In [141]:
cat_cols=[
    'prime_genre',
    'currency'
]

target_cols = ['user_rating']

num_cols = [
    'size_bytes',
    'price',
    'rating_count_tot',
    'rating_count_ver',
    'sup_devices.num',
    'ipadSc_urls.num',
    'lang.num',
    # Эта фича - не числовая, а порядковая, но мы все равно возьмем ее как числовую для удобства
    'cont_rating',
]

cols = num_cols + target_cols + cat_cols

In [142]:
data = data[cols]
# Возраст записан не в виде числа, исправим это, вырезав последний символ и скастовав к числу
data['cont_rating'] = data['cont_rating'].str.slice(0, -1).astype(int)
data.shape

(7197, 11)

In [143]:
data.isna().mean()

size_bytes          0.0
price               0.0
rating_count_tot    0.0
rating_count_ver    0.0
sup_devices.num     0.0
ipadSc_urls.num     0.0
lang.num            0.0
cont_rating         0.0
user_rating         0.0
prime_genre         0.0
currency            0.0
dtype: float64

In [144]:
for col in cat_cols:
    print(f"{col} DISTRIBUTION")
    print(data[col].value_counts())
    print()

prime_genre DISTRIBUTION
prime_genre
Games                3862
Entertainment         535
Education             453
Photo & Video         349
Utilities             248
Health & Fitness      180
Productivity          178
Social Networking     167
Lifestyle             144
Music                 138
Shopping              122
Sports                114
Book                  112
Finance               104
Travel                 81
News                   75
Weather                72
Reference              64
Food & Drink           63
Business               57
Navigation             46
Medical                23
Catalogs               10
Name: count, dtype: int64

currency DISTRIBUTION
currency
USD    7197
Name: count, dtype: int64



In [145]:
data = data.drop(columns=['currency'])
cat_cols.remove('currency')

In [146]:
data.corr(numeric_only=True).style.background_gradient(cmap='coolwarm')
# data.head()

,size_bytes,price,rating_count_tot,rating_count_ver,sup_devices.num,ipadSc_urls.num,lang.num,cont_rating,user_rating
size_bytes,1.000000,0.182392,0.004486,0.006337,-0.118347,0.152697,0.004614,0.140541,0.066256
price,0.182392,1.000000,-0.039044,-0.018012,-0.115361,0.066100,-0.006713,-0.029666,0.046601
rating_count_tot,0.004486,-0.039044,1.000000,0.163645,0.008832,0.015734,0.137675,0.009139,0.083310
rating_count_ver,0.006337,-0.018012,0.163645,1.000000,0.037951,0.024333,0.013287,0.001435,0.068754
sup_devices.num,-0.118347,-0.115361,0.008832,0.037951,1.000000,-0.037728,-0.041681,0.020085,-0.042451
ipadSc_urls.num,0.152697,0.066100,0.015734,0.024333,-0.037728,1.000000,0.088378,-0.107726,0.265671
lang.num,0.004614,-0.006713,0.137675,0.013287,-0.041681,0.088378,1.000000,-0.069508,0.170976
cont_rating,0.140541,-0.029666,0.009139,0.001435,0.020085,-0.107726,-0.069508,1.000000,-0.097999
user_rating,0.066256,0.046601,0.083310,0.068754,-0.042451,0.265671,0.170976,-0.097999,1.000000


## Подготовка данных
### Создание фичей

In [147]:
# Добавим категориальную фичу, которая говорит, бесплатное приложение или нет
data['is_free'] = data['price'] == 0
cat_cols.append('is_free')
data.head()

,size_bytes,price,rating_count_tot,rating_count_ver,sup_devices.num,ipadSc_urls.num,lang.num,cont_rating,user_rating,prime_genre,is_free
0,100788224,3.99,21292,26,38,5,10,4,4.0,Games,False
1,158578688,0.00,161065,26,37,5,23,4,4.0,Productivity,True
2,100524032,0.00,188583,2822,37,5,3,4,3.5,Weather,True
3,128512000,0.00,262241,649,37,5,9,12,4.0,Shopping,True
4,92774400,0.00,985920,5320,37,5,45,4,4.5,Reference,True


### Работа с категориальными признаками
#### One-hot-encoding
Пример:
Представьте, что у нас есть категориальная фича prime_genre с возможными значениями
> ['Games', 'Entertainment', 'Education', 'Photo & Video']

мы можем создать 4 новые бинарные фичи для каждого из столбцов
> 'Entertaiment' -> [0, 1, 0, 0]


In [148]:
a = pd.DataFrame({'categoria' : ['a', 'b', 'c']})
a

,categoria
0,a
1,b
2,c


In [149]:
pd.get_dummies(a, dtype=int)

,categoria_a,categoria_b,categoria_c
0,1,0,0
1,0,1,0
2,0,0,1


In [150]:
data = pd.get_dummies(data, columns=cat_cols)
# data
print(cat_cols)

['prime_genre', 'is_free']


In [155]:
cat_cols_new = []
# print(cat_cols)
for col_name in cat_cols:
    cat_cols_new.extend(filter(lambda x: x.startswith(col_name), data.columns))
    
cat_cols = cat_cols_new
print(cat_cols_new)


['prime_genre_Book', 'prime_genre_Business', 'prime_genre_Catalogs', 'prime_genre_Education', 'prime_genre_Entertainment', 'prime_genre_Finance', 'prime_genre_Food & Drink', 'prime_genre_Games', 'prime_genre_Health & Fitness', 'prime_genre_Lifestyle', 'prime_genre_Medical', 'prime_genre_Music', 'prime_genre_Navigation', 'prime_genre_News', 'prime_genre_Photo & Video', 'prime_genre_Productivity', 'prime_genre_Reference', 'prime_genre_Shopping', 'prime_genre_Social Networking', 'prime_genre_Sports', 'prime_genre_Travel', 'prime_genre_Utilities', 'prime_genre_Weather', 'is_free_False', 'is_free_True']


### Масштабирование
Процесс приведения всех признаков или независимых переменных (переменных, отличных от целевой переменной) к одному масштабу, чтобы каждый признак был одинаково важен.
#### Методы масштабирования:
1. Стандартизация
2. Нормализация min-max
3. итд


In [152]:
from sklearn.preprocessing import StandardScaler

df = data.copy()
std_scaler = StandardScaler()
data_scaled = std_scaler.fit_transform(pd.DataFrame(df['rating_count_tot']).to_numpy())
df['rating_count_tot'] = pd.DataFrame(data_scaled, columns=['rating_count_tot'])
df.head()
# data.head()

,size_bytes,price,rating_count_tot,rating_count_ver,sup_devices.num,ipadSc_urls.num,lang.num,cont_rating,user_rating,prime_genre_Book,...,prime_genre_Productivity,prime_genre_Reference,prime_genre_Shopping,prime_genre_Social Networking,prime_genre_Sports,prime_genre_Travel,prime_genre_Utilities,prime_genre_Weather,is_free_False,is_free_True
0,100788224,3.99,0.110902,26,38,5,10,4,4.0,False,...,False,False,False,False,False,False,False,False,True,False
1,158578688,0.00,1.956477,26,37,5,23,4,4.0,False,...,True,False,False,False,False,False,False,False,False,True
2,100524032,0.00,2.319827,2822,37,5,3,4,3.5,False,...,False,False,False,False,False,False,False,True,False,True
3,128512000,0.00,3.292413,649,37,5,9,12,4.0,False,...,False,False,True,False,False,False,False,False,False,True
4,92774400,0.00,12.847931,5320,37,5,45,4,4.5,False,...,False,True,False,False,False,False,False,False,False,True


## Разделение на train/test

In [153]:
from sklearn.model_selection import train_test_split

In [156]:
X = data[num_cols + cat_cols]
X_train, X_test, y_train, y_test=train_test_split(X,data[target_cols],train_size=0.7,random_state=43,stratify=data[target_cols])
print(f"X_train\n{X_train}")
print(f"y_train\n{y_train}")
print(f"X_test\n{X_test}")
print(f"y_test\n{y_test}")

X_train
      size_bytes  price  rating_count_tot  rating_count_ver  sup_devices.num  \
4502    57072640   0.00               343               130               43   
6522   208609280   0.00                86                 0               40   
6902    77396992   0.00               197                88               40   
3180   393954304   0.00             87898              8763               43   
1367   245444608   2.99              3389               203               40   
...          ...    ...               ...               ...              ...   
2807    53571584   2.99                 0                 0               26   
6867    43315200   0.00                27                 3               37   
5028   774950912   6.99                88                88               24   
4720    34132992   0.00                 0                 0               38   
1566   188902400   0.00             46363               213               37   

      ipadSc_urls.num  lang.num

## Обучение

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error

### Метрика
Метрика это, так же как и функция потерь, какая-то функция, показывающая насколько хорошо работает наша модель.Метрики нужны как раз для того, чтобы оценивать работу алгоритма с помощью числа, которое он не учился напрямую минимизировать.

В данном случае у нас задача регрессии, поэтому мы используем две метрики MSE и R_squared.

$$R^2 = 1 - \frac{\sum_{i=1}^{n} (y^i - y_{pred}^i)^2}{\sum_{i=1}^{n} (y^i - y_{mean})^2}$$

R_aquared это доля объясненной вариации. R_squared = 1, когда у нас есть идеальный предсказатель. R_squared = 0 достигает модель, которая просто выдает в качестве ответа среднее Значение целевой переменной. А значения меньше 0 говорят о том, что модель хуже константного предсказателя.

In [ ]:
def print_metrics(y_preds, y):
    print(f'R^2: {r2_score(y_preds, y)}')
    print(f'MSE: {mean_squared_error(y_preds, y)}')

In [ ]:
# Используем обычную линейную регрессию, минимизирующую сумму квадратов ошибки
lr = LinearRegression()
lr.fit(X_train, y_train)

print_metrics(lr.predict(X_test), y_test)

R^2: -5.04963222211981
MSE: 2.0299934812029954


In [ ]:
# Используем обычную линейную регрессию, минимизирующую сумму квадратов ошибки
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

print_metrics(knn.predict(X_test), y_test)

R^2: -4.0963893707046894
MSE: 2.7014722222222223


## Cross Validation
<center><img src="cross.jpg" width=700 style="display: inline-block;"></center>


Выберем 
 - количество частей, на которые разобьется наш датасет
 - for i = 1...k :
    - Обучим модель на всех частях датасета, кроме i-ой.
    - Посчитаем метрики или предсказания для i-ой части
 - Саггрегируем все все предсказания или усредним метрики

Таким образом мы сможем получить более объективные предсказания нашей модели, использовав весь датасет как train и как test, при этом не создав утечек данных.

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate

In [ ]:
cross_validate(LinearRegression(), X, data[target_cols], cv=5,
               scoring={'r2_score': make_scorer(r2_score),
                        'mean_squared_error': make_scorer(mean_squared_error)})

{'fit_time': array([0.01395345, 0.01196122, 0.01295376, 0.01011586, 0.01195526]),
 'score_time': array([0.0049839 , 0.00498152, 0.00498343, 0.01483917, 0.00897193]),
 'test_r2_score': array([-0.83180594,  0.06616553,  0.11575665,  0.10998759,  0.07104445]),
 'test_mean_squared_error': array([1.66689747, 1.48766359, 1.9403641 , 2.81734145, 2.93209146])}

In [ ]:
cross_validate(KNeighborsRegressor(), X, data[target_cols], cv=5,
               scoring={'r2_score': make_scorer(r2_score, ),
                        'mean_squared_error': make_scorer(mean_squared_error)})

{'fit_time': array([0.0069778 , 0.00896907, 0.00697756, 0.00596476, 0.00652528]),
 'score_time': array([0.04484868, 0.04959655, 0.04426503, 0.04584575, 0.04533529]),
 'test_r2_score': array([-0.99851356, -0.29720715, -0.17219446, -0.1199438 , -0.15899505]),
 'test_mean_squared_error': array([1.81859722, 2.06654167, 2.57223767, 3.5451911 , 3.65817234])}